In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/binance-full-history/WAVES-USDT.parquet
/kaggle/input/binance-full-history/RLC-BNB.parquet
/kaggle/input/binance-full-history/AE-BTC.parquet
/kaggle/input/binance-full-history/BTC-GBP.parquet
/kaggle/input/binance-full-history/RVN-BNB.parquet
/kaggle/input/binance-full-history/DOTDOWN-USDT.parquet
/kaggle/input/binance-full-history/MATIC-BUSD.parquet
/kaggle/input/binance-full-history/HIVE-BTC.parquet
/kaggle/input/binance-full-history/DOT-BUSD.parquet
/kaggle/input/binance-full-history/XRP-USDC.parquet
/kaggle/input/binance-full-history/KAVA-USDT.parquet
/kaggle/input/binance-full-history/AXS-USDT.parquet
/kaggle/input/binance-full-history/STX-BTC.parquet
/kaggle/input/binance-full-history/XRP-PAX.parquet
/kaggle/input/binance-full-history/SHIB-BUSD.parquet
/kaggle/input/binance-full-history/CTXC-USDT.parquet
/kaggle/input/binance-full-history/TRX-USDT.parquet
/kaggle/input/binance-full-history/ANKR-BTC.parquet
/kaggle/input/binance-full-history/DOT-TRY.parquet
/kaggle/i

In [2]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from sklearn.metrics import classification_report
import pyarrow.parquet as pq
#import talib as ta
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras import metrics
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, TimeDistributed, Flatten, MaxPooling1D, MaxPooling2D, InputLayer, BatchNormalization, Conv2D
from tensorflow.keras import optimizers
import random

Using TensorFlow backend.


In [3]:
dataset = pq.read_pandas("/kaggle/input/binance-full-history/BTC-USDT.parquet").to_pandas()
dataset.sort_values("open_time")
dataset = dataset.resample("4h").apply(dict(open="first", high="max", low="min", close="last"))
dataset = dataset.dropna()
dataset

,open,high,low,close
open_time,,,,
2017-08-17 04:00:00,4261.479980,4349.990234,4261.319824,4349.990234
2017-08-17 08:00:00,4333.319824,4485.390137,4333.319824,4427.299805
2017-08-17 12:00:00,4427.299805,4485.390137,4333.419922,4352.339844
2017-08-17 16:00:00,4352.339844,4354.839844,4200.740234,4325.229980
2017-08-17 20:00:00,4307.560059,4369.689941,4258.560059,4285.080078
...,...,...,...,...
2022-02-07 04:00:00,42819.000000,43000.000000,42551.960938,42723.921875
2022-02-07 08:00:00,42723.910156,42767.691406,42234.089844,42720.031250
2022-02-07 12:00:00,42720.031250,43895.019531,42490.019531,43835.589844


In [4]:
def preprocess(df, window_size, valid):
  df = df.values
  buys = []
  holds = []
  sequences = []
  def prepare_batches(batch):
    label = batch[-1][-1]
    close = batch[-1][-2]
    low = batch[-1][-3]
    high = batch[-1][-4]
    batch_without_label = []
    for b in batch:
        b = np.delete(b, -1)
        batch_without_label.append(b)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_batch = scaler.fit_transform(batch_without_label)
    if label == 1:
        buys.append([scaled_batch, close, high, low, label])
    else:
        holds.append([scaled_batch, close, high, low, label])
    sequences.append([scaled_batch, close, high, low, label])
    return 1
    
  shape = (df.shape[0] - window_size + 1, window_size) + df.shape[1:]
  strides = (df.strides[0],) + df.strides
  batched_df = np.lib.stride_tricks.as_strided(df, shape=shape, strides=strides)
  mapped = list(map(prepare_batches, batched_df))
  
  if not valid:
      print("Buys: ", len(buys))
      print("Holds: ", len(holds))
      lower = min(len(buys), len(holds))
      buys = buys[-lower:]
      holds = holds[-lower:]
      print("New Buys: ", len(buys))
      print("New Holds: ", len(holds))
      print("-----------------------")
      sequences = buys + holds
      random.shuffle(sequences)

  X = []
  y = []
  c = []
  h = []
  l = []
  def seperate_sequences(sequence):
    X.append(sequence[0])
    c.append(sequence[1])
    h.append(sequence[2])
    l.append(sequence[3])
    y.append(sequence[4])
    return 1

  mapped = list(map(seperate_sequences, sequences))
  results = pd.DataFrame()
  results["close"] = c
  results["high"] = h
  results["low"] = l
  results["label"] = y
  return np.array(X), np.array(y), results

In [5]:
def labelize(df, sl, tp, candle_interval):
    df["sl"] = df.close*(1-(sl/100))
    df["tp"] = df.close*(1+(tp/100))

    df["tp_candle"] = 0
    df["sl_candle"] = candle_interval+2
    for i in range(1, candle_interval+1):
        df["prev_max_"+str(i)] = df.high.rolling(window=i).max()
        df["next_max_"+str(i)] = df["prev_max_"+str(i)].shift(-i)
        df["tp_candle"] = np.where((df["tp_candle"]==0)&(df["next_max_"+str(i)]>=df["tp"]), i, df["tp_candle"])

        df["prev_min_"+str(i)] = df.low.rolling(window=i).min()
        df["next_min_"+str(i)] = df["prev_min_"+str(i)].shift(-i)
        df["sl_candle"] = np.where((df["sl_candle"]==candle_interval+2)&(df["next_min_"+str(i)]<=df["sl"]), i, df["sl_candle"])

        df = df.drop(["next_max_"+str(i), "next_min_"+str(i), "prev_max_"+str(i), "prev_min_"+str(i)], axis=1)

    df["label"] = np.where((df["tp_candle"]!=0)&(df["tp_candle"]<df["sl_candle"]), 1, 0)
    #df = df.drop(["sl", "tp", "sl_candle", "tp_candle"], axis=1)
    print(f"{len(df.loc[df.label==1])} / {len(df)}")
    return df

In [6]:
df = dataset.copy()
df = labelize(df, 1, 2, 240)
df

2936 / 9797


,open,high,low,close,sl,tp,tp_candle,sl_candle,label
open_time,,,,,,,,,
2017-08-17 04:00:00,4261.479980,4349.990234,4261.319824,4349.990234,4306.490234,4436.989746,1,3,1
2017-08-17 08:00:00,4333.319824,4485.390137,4333.319824,4427.299805,4383.026855,4515.845703,73,1,0
2017-08-17 12:00:00,4427.299805,4485.390137,4333.419922,4352.339844,4308.816406,4439.386719,49,1,0
2017-08-17 16:00:00,4352.339844,4354.839844,4200.740234,4325.229980,4281.977539,4411.734375,47,1,0
2017-08-17 20:00:00,4307.560059,4369.689941,4258.560059,4285.080078,4242.229492,4370.781738,3,1,0
...,...,...,...,...,...,...,...,...,...
2022-02-07 04:00:00,42819.000000,43000.000000,42551.960938,42723.921875,42296.683594,43578.398438,2,1,0
2022-02-07 08:00:00,42723.910156,42767.691406,42234.089844,42720.031250,42292.832031,43574.429688,1,242,1
2022-02-07 12:00:00,42720.031250,43895.019531,42490.019531,43835.589844,43397.234375,44712.300781,0,242,0


In [7]:
train = df.iloc[:int(len(df)*0.9)]
test = df.iloc[int(len(df)*0.7):]
x_train, y_train, _ = preprocess(train, 1, False)
x_test, y_test, results = preprocess(test, 1, True)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(results.shape)

Buys:  2666
Holds:  6151
New Buys:  2666
New Holds:  2666
-----------------------
(5332, 1, 8)
(5332,)
(2940, 1, 8)
(2940,)
(2940, 4)


In [8]:
model = Sequential()

model.add(InputLayer(input_shape=x_train.shape[1:]))

model.add(Conv1D(128, 60, padding="same", activation="relu"))
model.add(Dropout(0.2))

model.add(Conv1D(128, 60, padding="same", activation="relu"))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(BatchNormalization())

model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1, 128)            61568     
_________________________________________________________________
dropout (Dropout)            (None, 1, 128)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 128)            983168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
dense (Dense)                (None, 512)               6

In [9]:
history = model.fit(x_train, y_train, shuffle=True, validation_split=0.1, batch_size=32, epochs=10, verbose=1)

Train on 4798 samples, validate on 534 samples
Epoch 1/10
4798/4798 [==============================] - 12s 3ms/sample - loss: 0.6932 - acc: 0.5002 - val_loss: 0.6933 - val_acc: 0.4719
Epoch 2/10
4798/4798 [==============================] - 11s 2ms/sample - loss: 0.6932 - acc: 0.5031 - val_loss: 0.6933 - val_acc: 0.4719
Epoch 3/10
4798/4798 [==============================] - 11s 2ms/sample - loss: 0.6932 - acc: 0.5031 - val_loss: 0.6934 - val_acc: 0.4719
Epoch 4/10
4798/4798 [==============================] - 11s 2ms/sample - loss: 0.6932 - acc: 0.5031 - val_loss: 0.6934 - val_acc: 0.4719
Epoch 5/10
4798/4798 [==============================] - 11s 2ms/sample - loss: 0.6932 - acc: 0.5031 - val_loss: 0.6935 - val_acc: 0.4719
Epoch 6/10
4798/4798 [==============================] - 11s 2ms/sample - loss: 0.6932 - acc: 0.4927 - val_loss: 0.6935 - val_acc: 0.4719
Epoch 7/10
4798/4798 [==============================] - 11s 2ms/sample - loss: 0.6932 - acc: 0.5031 - val_loss: 0.6936 - val_acc: 0

In [10]:
predictions = model.predict(x_test)
binary_predictions = np.round(predictions).astype(int)
print(classification_report(y_test, binary_predictions, target_names=["HOLD", "LONG"]))

              precision    recall  f1-score   support

        HOLD       0.00      0.00      0.00      2049
        LONG       0.30      1.00      0.47       891

    accuracy                           0.30      2940
   macro avg       0.15      0.50      0.23      2940
weighted avg       0.09      0.30      0.14      2940



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
